In [53]:
import cirq
import sympy
import tensorflow_quantum as tfq
import tensorflow as tf
import numpy as np
from cirq import Simulator

In [54]:
def HEA(qubits,Nlayer,parameters):
    circuit=cirq.Circuit()
    count=0
    Nqubits=len(qubits)
    for i in range(Nlayer):  
        for i in range(Nqubits):
            circuit.append(cirq.ry(parameters[count])(qubits[i]))
            count+=1
        for i in range(Nqubits):
            circuit.append(cirq.rz(parameters[count])(qubits[i]))
            count+=1
        for i in range(1,Nqubits-1,2):
           circuit.append(cirq.CNOT(qubits[i],qubits[i+1])) 
        for i in range(0,Nqubits,2):
            circuit.append(cirq.CNOT(qubits[i],qubits[i+1]))
    for i in range(Nqubits):
        circuit.append(cirq.ry(parameters[count])(qubits[i]))
        count+=1
    for i in range(Nqubits):
        circuit.append(cirq.rz(parameters[count])(qubits[i]))
        count+=1
    return circuit

In [55]:
def hamiltonian(qubits):
    h = [0]
    for i in range(len(qubits)-1):
        h.append(cirq.X(qubits[i])*cirq.X(qubits[i+1]))
        h.append(cirq.Y(qubits[i])*cirq.Y(qubits[i+1]))
        h.append(cirq.Z(qubits[i])*cirq.Z(qubits[i+1]))
    return h
def Param(theta,params):
    p={}
    for i in range(len(params)):
        p[params[i]]=theta[i]
    return cirq.ParamResolver(p)

In [56]:
Nqubits=4
Nlayer=1
Nparams=Nqubits*(Nlayer+1)*2
qubits=[cirq.GridQubit(i,0) for i in range(Nqubits)]
params=sympy.symbols('vqe0:%d'%Nparams)

In [57]:
circuit=HEA(qubits,Nlayer,params)
observable = sum(hamiltonian(qubits))

In [58]:
#using QTF to calculate the optimal parameters.
init = tf.keras.initializers.RandomUniform(-np.pi, np.pi)
ins = tf.keras.Input(shape=(), dtype=tf.dtypes.string)
inputs = tfq.convert_to_tensor([cirq.Circuit()])
outs = tfq.layers.PQC(circuit, observable, differentiator=tfq.differentiators.ParameterShift(), initializer=init)(ins)
vqe = tf.keras.models.Model(inputs=ins, outputs=outs)
opt = tf.keras.optimizers.Adam(lr=0.005)
vqe.compile(loss=tf.keras.losses.MAE, optimizer=opt)
expt_gd=[]
for index in range(0,100):
    with tf.GradientTape() as tape:
        loss=vqe(inputs)
    gradients=tape.gradient(loss,vqe.trainable_variables)
    opt.apply_gradients(zip(gradients,vqe.trainable_variables))
    expt_gd.append(vqe(inputs).numpy()[0][0])

In [59]:
theta=vqe.trainable_variables[0].numpy()

#reset_weights to vqe to get the estimation energy value
vqe.set_weights([theta])
rwqv=vqe(inputs)

#using tfq.layers.Expectation to get the estimation energy value
params_str=['vqe'+str(i) for i in range(Nparams)]
qev=tfq.layers.Expectation()(circuit,
                         symbol_names=params_str,
                         symbol_values=[theta],
                         operators=observable)

#using simulator to get the estimation energy value.
simulator=Simulator()
resolver=Param(theta,params) #参数器 
sv=simulator.simulate_expectation_values(circuit,observable,resolver)


print("parameters_number:{npara}, QTF_value:{qv}, Reset_Weights_QTF_Value:{rwqv}, QTF.Expectation_value:{qev}, Simulator_value:{sv}".format(npara=Nparams,qv=expt_gd[-1], rwqv=rwqv, qev=qev, sv=sv))

parameters_number:16, QTF_value:-4.257663726806641, Reset_Weights_QTF_Value:[[-4.2576637]], QTF.Expectation_value:[[-1.1815777]], Simulator_value:[(-1.1815786734223366+7.450580596923828e-09j)]
